In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
class Tokenizer():
    def __init__(self, file_path):
        self.sonnets = []
        self.vocab = self.create_vocab(file_path)
        self.stoi, self.itos = self.create_dict()
        self.file_path = file_path
        self.max_len = max([len(i) for i in self.sonnets])
        self.indexes = torch.arange(len(self.vocab))

    def create_vocab(self, path):
        f = open(path)
        vocab_set = {'<SOS>', '<EOS>'}
        sonnet = []
        for line in f.readlines():
            try:
                int(line.strip())
                if sonnet:
                    sonnet = ['<SOS>'] + sonnet + ['<EOS>']
                    self.sonnets.append(sonnet)
                sonnet = []
            except ValueError:
                words = [i.lower().strip() for i in line.split()]
                sonnet += (words)
                vocab_set.update(set(words))
        sonnet = ['<SOS>'] + sonnet + ['<EOS>']
        self.sonnets.append(sonnet)
        return vocab_set

    def create_dict(self):
        stoi = OrderedDict()
        itos = OrderedDict()
        for i, token in enumerate(self.vocab):
            stoi[token] = i
            itos[i] = token
        return stoi, itos

    def sonnet_to_tensor(self, sonnet):
        x = torch.zeros(self.max_len, len(self.vocab))
        for idx, word in enumerate(sonnet):
            x[idx, self.stoi[word]] = 1
        return x

    def get_tensor_data(self):
        data_tensor = torch.zeros(len(self.sonnets), self.max_len, len(self.vocab))
        for idx, sonnet in enumerate(self.sonnets):
            data_tensor[idx] = self.sonnet_to_tensor(sonnet)
        return data_tensor

    def word_to_one_hot(self, index, batch_size=1):
        x = torch.zeros(batch_size, 1, len(self.vocab))
        x[:, :, index] = 1
        return x

    def word_to_one_hot_batch(self, index):
        x = torch.zeros(index.shape[0], 1, len(self.vocab))
        for i, idx in enumerate(index):
            x[i] = self.word_to_one_hot(idx)  
        return x
        
    def decode(self, sonnet):
        words = ''
        try:
            for idx, row in enumerate(sonnet):
                words += self.itos[self.indexes[row.to(torch.bool)].item()] + ' '        
        except RuntimeError:
            pass
        return words

<strong> Recurrent Neural Networks</strong>
<p>  Hidden state update : </p> $$h_t = tanh(x_tW^T_{ih} + b_{ih} + h_{t-1}W^T_{hh} + b_{hh})$$
<p>$x_t$ : input at time step t</p>
<p>$W_{ih}$: input to hidden weight matrix</p>
<p>$b_{ih}$: input to hidden biases</p>
<p>$h_{t-1}$: previous hidden state</p>
<p>$W_{hh}$: weight matrix for previous hidden state to current hidden state</p>
<p>$h_t$: current hidden state</p>
<br>

<strong> Gated Recurrent Unit</strong>
\begin{align*}
r_t &= \sigma(W_{ir} x_t + b_{ir} + W_{hr} h_{(t-1)} + b_{hr}) \\
z_t &= \sigma(W_{iz} x_t + b_{iz} + W_{hz} h_{(t-1)} + b_{hz}) \\
n_t &= \tanh(W_{in} x_t + b_{in} + r_t \odot (W_{hn} h_{(t-1)} + b_{hn})) \\
h_t &= (1 - z_t) \odot n_t + z_t \odot h_{(t-1)}
\end{align*}

<p>$x_t$ : input at time step t</p>
<p>$h_{t-1}$: previous hidden state</p>
<p>$h_t$: current hidden state</p>
<p>$r_t$: reset gate</p>
<p>$z_t$: update gate</p>
<p>$n_t$: candidate for current hidden state</p>
<p>$h_t$: current hidden state</p>

<strong> Long short term memory</strong>
<p>Hidden state update :</p>

\begin{align*}
\Gamma_u &= \sigma(W_{iu} x_t + b_{iu} + W_{hu} h_{t-1} + b_{hu}) \\
\Gamma_f &= \sigma(W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf}) \\
\tilde{c_t} &= \tanh(W_{ic} x_t + b_{ic} + W_{hc} h_{t-1} + b_{hc}) \\
\Gamma_o &= \sigma(W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho}) \\
c_t &= \Gamma_f \odot c_{t-1} + \Gamma_u \odot \tilde{c_t} \\
h_t &= \Gamma_o \odot \tanh(c_t)
\end{align*}

<p>$x_t$ : input at time step t</p>
<p>$h_{t-1}$: previous hidden state</p>
<p>$h_t$: current hidden state</p>
<p>$\Gamma_u$: update gate</p>
<p>$\Gamma_o$: output gate</p>
<p>$\Gamma_f$: forget gate</p>
<p>$\tilde{c_t} $: candidate for current memory state</p>
<p>$c_t$: current memory cell state</p>

In [3]:
tokenizer = Tokenizer('shakespeare.txt')

In [4]:
data_tensor = tokenizer.get_tensor_data()

In [5]:
class SonnetDataset(Dataset):
    def __init__(self, x):
        self.x = x
        self.samples = x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx]

    def __len__(self):
        return self.samples

In [6]:
train_dataset = SonnetDataset(data_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [7]:
class SeqGeneration(nn.Module):
    def __init__(self, input_size, hidden_size, num_cells, vocab_length, cell_init='rnn', bidirectional=False):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_cells = num_cells
        self.num_directions = 2 if bidirectional else 1
        self.cell_init = cell_init
        if cell_init == 'rnn':
            self.cell = nn.RNN(input_size, hidden_size, num_cells, batch_first=True, bidirectional=bidirectional, dropout=0.2)
        elif cell_init == 'gru':
            self.cell = nn.GRU(input_size, hidden_size, num_cells, batch_first=True, bidirectional=bidirectional, dropout=0.2)
        elif cell_init == 'lstm':
            self.cell = nn.LSTM(input_size, hidden_size, num_cells, batch_first=True, bidirectional=bidirectional, dropout=0.2)
        else:
            raise Exception('Invalid cell type')
        self.fc1 = nn.Linear(self.num_directions*hidden_size, vocab_length)
        self.softmax = nn.LogSoftmax(dim=-1)
        

    def forward(self, x, h0=None, c0=None, device='mps'):
        # data you pass will be the first time step only.
        if h0 is None:
            h0 = torch.zeros(self.num_directions*self.num_cells, x.shape[0], self.hidden_size, device=device)
        if self.cell_init == 'lstm':
            if c0 is None:
                c0 = torch.zeros(self.num_directions*self.num_cells, x.shape[0], self.hidden_size, device=device)
            hidden_states, (ht_all, c0) = self.cell(x, (h0, c0))
        else:
            hidden_states, ht_all = self.cell(x, h0)
            # ht will always be of the shape h0 
        ht = hidden_states[:, -1, :]
        logits = self.fc1(ht)
        scores = self.softmax(logits)
        return scores, ht_all, c0

In [8]:
model = SeqGeneration(len(tokenizer.vocab), 128, 2, len(tokenizer.vocab), 'lstm', True)
# model.load_state_dict(torch.load('SeqGeneration.pth'))
device = 'mps'
model = model.to(device)
loss = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [9]:
def train(model, train_loader, train_params, tokenizer):
    min_loss = float('inf')
    epochs = train_params['epochs']
    loss = train_params['loss']
    optimizer = train_params['optimizer']
    for epoch in range(epochs):
        loss_at_batch = None
        for x in tqdm(train_loader, desc=f'Epoch: {epoch}/{epochs}, best loss: {min_loss}'):
            loss_at_batch = []
            ht = None
            c0 = None
            x = x.to(device)
            for t in range(x.shape[1] - 1):
                xt = x[:, t:t+1, :]
                scores, ht, c0 = model(xt, ht, c0)
                next_word_index = torch.argmax(scores, dim=1)
                labels = x[:, t+1:t+2, :]
                labels = torch.argmax(labels, dim=2).flatten()
                if labels.sum() == 0:
                    break
                loss_at_timestep = loss(scores, labels)
                loss_at_batch.append(loss_at_timestep)
                xt = tokenizer.word_to_one_hot_batch(next_word_index)
                xt = xt.to(device)
            loss_at_batch = sum(loss_at_batch)/len(loss_at_batch)
            optimizer.zero_grad()
            loss_at_batch.backward()
            torch.nn.utils.clip_grad_value_(model.parameters(), 0.5)
            optimizer.step()
        if loss_at_batch < min_loss:
            torch.save(model.state_dict(), 'SeqGeneration_lstm.pth')
            min_loss = loss_at_batch
            print('saving model')

In [10]:
def generate_text(model, sos_tensor, tokenizer):
    curr_token = ''
    sonnet = ''
    pred = ''
    ht = None
    model.eval()
    with torch.no_grad():  
        while curr_token != '<EOS>' and len(sonnet.split()) < tokenizer.max_len:
            sonnet += pred + ' '
            scores, ht, ct = model(sos_tensor, ht, None, 'cpu')
            # pred = tokenizer.itos[torch.argmax(scores).item()] always generates the same sequence
            # sample to generate different sequences each time.
            pred = tokenizer.itos[torch.argmax(scores).item()]
            random_word_index = torch.multinomial(torch.exp(scores).flatten(), num_samples=1).item()
            curr_token = pred
            sos_tensor = tokenizer.word_to_one_hot(tokenizer.stoi[tokenizer.itos[random_word_index]], 1)
        return sonnet

In [11]:
train_params = {'epochs': 200, 'loss': loss, 'optimizer': optimizer} # set epochs to train
train(model, train_loader, train_params, tokenizer)

Epoch: 0/200, best loss: inf: 100%|█████████████| 10/10 [00:07<00:00,  1.41it/s]


saving model


Epoch: 1/200, best loss: 7.480195045471191: 100%|█| 10/10 [00:06<00:00,  1.52it/


saving model


Epoch: 2/200, best loss: 6.760316848754883: 100%|█| 10/10 [00:06<00:00,  1.54it/


...

saving model


Epoch: 198/200, best loss: 1.7870066165924072: 100%|█| 10/10 [00:07<00:00,  1.40


saving model


Epoch: 199/200, best loss: 1.769547939300537: 100%|█| 10/10 [00:06<00:00,  1.47i

saving model


In [12]:
model.load_state_dict(torch.load('SeqGeneration_lstm.pth'))  # with lstm
model.to('cpu') 
generate_text(model, tokenizer.word_to_one_hot(tokenizer.stoi['<SOS>'], 1), tokenizer)

' so centre when nothing power not beauty want all power and the fever his be subject and thou fear a time lovely me my lovely what gentle thou nothing birth, to did supposing thee thou live, want me winter check thee will, be his the power once live, i will, thee live, and wish, the lovely not fear and fever of live, were me creatures of heart of time the power and live, the power live, thee thou centre thee creatures thee will, of fear lovely with will, blamed thee heart substance her beauty thee time of the old want that live, power journey to journey and force, hath show creatures thee as lovely to my lovely with heart power nothing me gentle of love, check not live, to time creatures me '

In [13]:
model = SeqGeneration(len(tokenizer.vocab), 128, 1, len(tokenizer.vocab), 'rnn', True)  #with rnns
model.load_state_dict(torch.load('SeqGeneration.pth'))
model.to('cpu')
generate_text(model, tokenizer.word_to_one_hot(tokenizer.stoi['<SOS>'], 1), tokenizer)

/Users/yasodasairamkandikonda/Desktop/Pytorch/env/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


" devouring wond'ring prophecies hid wond'ring wond'ring well-contented unknown, very barrenly fire barrenly rough heaven: therefore endowed, lend least o'erpressed wont ceremony wond'ring stars moan prophecies out. deserts seeing moan crown devise. power disgrace. sweets: prophecies faster nor endowed, go, glad their noon: sullen stars budding contend. weed, fed, endowed, poor crown wooing celestial ward, wailing world, preserve preserve preserve preserve preserve of stop stop world, preserve preserve preserve preserve of light stop world, preserve preserve preserve preserve preserve preserve world, preserve preserve preserve preserve preserve world, preserve preserve preserve preserve preserve preserve preserve preserve preserve preserve preserve preserve preserve preserve world, preserve world, preserve preserve preserve world, world, preserve preserve preserve preserve preserve world, that world, preserve preserve hopes, preserve merit? world, world, preserve world, preserve preser

<p> LSTMs were able to generate a <EOS> token before the specifed max limit.</p>
<p> RNNs however could not do this and relatively bad generation.</p>
<p> In the future, we will use <strong>Attention</strong> to make these models better. Attention module helps network focus on specific parts of sentence. This context helps model generate better sentences.</p>